In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


In [2]:
mnist_datasets, mnist_info = tfds.load(name= 'mnist' , with_info= True, as_supervised = True)

In [3]:
mnist_train, mnist_test = mnist_datasets['train'], mnist_datasets['test']

In [4]:
#Splitting the dataset into training, testing and validation dataset

num_validation_samples = 0.1* mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1* mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

#scaling the dataset

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)


In [5]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [6]:
#model outline

input_size = 784
output_size = 10
hidden_layer_size = 75

model = tf.keras.Sequential([ 
                              tf.keras.layers.Flatten(input_shape=(28,28,1)),
                              tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
                              tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
                              tf.keras.layers.Dense(output_size, activation= 'softmax')
                            ])

In [7]:
model.compile(optimizer= 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
#training the model

Num_epochs = 5

model.fit(train_data, epochs = Num_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3633 - accuracy: 0.8929 - val_loss: 0.1814 - val_accuracy: 0.9470
Epoch 2/5
540/540 - 2s - loss: 0.1569 - accuracy: 0.9533 - val_loss: 0.1341 - val_accuracy: 0.9593
Epoch 3/5
540/540 - 2s - loss: 0.1142 - accuracy: 0.9656 - val_loss: 0.1091 - val_accuracy: 0.9667
Epoch 4/5
540/540 - 2s - loss: 0.0908 - accuracy: 0.9731 - val_loss: 0.0822 - val_accuracy: 0.9763
Epoch 5/5
540/540 - 2s - loss: 0.0769 - accuracy: 0.9766 - val_loss: 0.0753 - val_accuracy: 0.9775


In [9]:
#testing the model

test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 0s 36ms/step - loss: 0.0950 - accuracy: 0.9710
